In [22]:
import openai
import json
from datetime import datetime, timedelta

with open('../api.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    openai.api_key = json_data["openai_api_key"]

In [10]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

In [13]:
user_prompt = "When's the next flight from Amsterdam to New York?"
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role": "user", 
            "content": user_prompt
        },
    ],     
    functions = function_descriptions,
    function_call = "auto",
)

output = response.choices[0].message
print(output)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_flight_info",
    "arguments": "{\n  \"loc_origin\": \"AMS\",\n  \"loc_destination\": \"JFK\"\n}"
  }
}


In [15]:
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""
    
    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }
    return json.dumps(flight_info)

In [16]:
origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)

print(origin)
print(destination)
print(params)

AMS
JFK
{'loc_origin': 'AMS', 'loc_destination': 'JFK'}


In [23]:
chosen_function = eval(output.function_call.name)
flight = chosen_function(**params) # ** to unpack parameters

In [29]:
print(chosen_function)
print(flight)

<function get_flight_info at 0x7f1d4ffc48b0>
{"loc_origin": "AMS", "loc_destination": "JFK", "datetime": "2023-07-27 17:03:14.716904", "airline": "KLM", "flight": "KL643"}


In [27]:
second_response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role": "user", 
            "content": user_prompt
        },
        {
            "role": "function",
            "name": output.function_call.name,
            "content": flight
        }
    ],     
    functions = function_descriptions,
)
response = second_response.choices[0].message.content
print(response)

The next flight from Amsterdam (AMS) to New York (JFK) is on July 27th, 2023 at 17:03. The airline is KLM and the flight number is KL643.


In [5]:
print(response)

{
  "id": "chatcmpl-7h2ynVwivlSie4mmr6BgdjvcFKUCU",
  "object": "chat.completion",
  "created": 1690493913,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I'm sorry, but as an AI language model, I do not have real-time information. Flight schedules can vary depending on the airline, date, and time of year. I recommend checking with airlines or using flight tracking websites or apps to find the most up-to-date information on flight schedules from Amsterdam to New York."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 18,
    "completion_tokens": 64,
    "total_tokens": 82
  }
}


In [13]:
reply_content = response.choices[0].message.content
reply_content

'How can I assist you today?'

In [14]:
message_history = []
user_input = input(">: ")

print("User's input was ", user_input)

User's input was  You're a helpful assistant


In [15]:
message_history.append({"role": "user", "content": user_input})